#### Install TensorRT

1. Go to: https://developer.nvidia.com/tensorrt.
2. Click Download.
3. Complete the TensorRT Download Survey.
4. Select the checkbox to agree to the license terms.
5. Click the package you want to install. Your download begins.

In [ ]:
import os
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt

from tensorflow.python.platform import gfile
from tensorflow.python.framework import graph_io

In [ ]:
big_model_input_dir    = 'model'
big_model_name         = 'tf_model.pb'
small_model_output_dir = 'model'
small_model_name       = 'tfrt_model.pb'

In [ ]:
output_names = ['filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0',
                'filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0',
                'filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0']

In [ ]:
big_model = os.path.join(big_model_input_dir, big_model_name)

In [ ]:
sess=tf.InteractiveSession()

f = gfile.FastGFile(big_model, 'rb')
graph_def = tf.GraphDef()
# Parses a serialized binary message into the current message.
graph_def.ParseFromString(f.read())
f.close()

sess.graph.as_default()
# Import a serialized TensorFlow `GraphDef` protocol buffer
# and place into the current default `Graph`.
tf.import_graph_def(graph_def)

In [ ]:
trt_graph = trt.create_inference_graph(
    input_graph_def=graph_def,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16',
    minimum_segment_size=50)

In [ ]:
graph_io.write_graph(trt_graph, 
                     small_model_output_dir, 
                     small_model_name, 
                     as_text=False)